In [27]:
import requests
import json
import pandas as pd
import random
import time
from datetime import datetime
from timeit import default_timer as timer
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS, ALL_REGIONS

## Functions

In [30]:
#Better readability for JSON output
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent = 4)
    print(text)
    
def get_trades(tid_start, tid_end, currency):
    
    import time
    if tid_start >= tid_end: #make sure valid starting and ending values provided
        tid = tid_start #initialize 'tid' to starting value
        tmp_list = [] #temp list to hold dfs from loop
        trades = pd.DataFrame() #final df of trades
        i = 0 #while-loop counter for sleep function
        
        gateway = ApiGateway("https://localbitcoins.com", regions = ["us-west-2"]) #Use ApiGateway from requests_ip_rotator to randomize IP used for requests to avoid rate limiting
        gateway.start() #open gateway, always shutdown as well

        session = requests.Session() #begin session
        session.mount("https://localbitcoins.com", gateway)

        
        try:
            while (tid >= tid_end):
                
                parameters = {
                    'max_tid': tid #passes current value of 'tid' as a parameter into the API request
                }

                url = 'https://localbitcoins.com/bitcoincharts/' + str(currency) + '/trades.json'

                t = session.get(url, params = parameters) #pull last 500 trades, starting with max_tid
                t.raise_for_status() #Returns HTTP error object
                df = pd.DataFrame(t.json()) #get JSON output into pd DataFrame
                tmp_list.append(df.iloc[:-1]) #append to temp list, omits last row to avoid repeat obs
                
                tid_last = df['tid'].iloc[-1] #gets the last tid of the previous dataset, new starting point for next loop
                tid = tid_last
                time.sleep(1 + (random.randint(0, 1000) / 1000))#sleep function to avoid API errors, this one moderates time between API requests
                i += 1 #loop counter
                
                if i%500 == 0: #only for multiples of 500 (get API error around 800 without IP randomization)
                    # time.sleep(1200) #sleep for 20 minutes
                    print('congrats! you have made ' + str(i) + ' API requests, you insane person.')
                
                
                
        except Exception as e:
            gateway.shutdown()
            print(t.status_code)
            print(e.response.text)
            print(tid)
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades
        
        else:
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades

    else:
        print('Please enter a value for tid_start that is greater than tid_end')

## Get list of currencies

In [4]:
altcoin = []
name = []

u = "https://localbitcoins.com/api/currencies/"
c = requests.get(u)
d = c.json()['data']['currencies'] #get to higher level of nesting
symbols = list(d.keys()) #get currency symbols

for currency in d:
    altcoin.append(d[str(currency)]['altcoin'])
    name.append(d[str(currency)]['name'])
    
currencies = pd.DataFrame(zip(symbols, name, altcoin), columns = ['symbol', 'currency', 'altcoin'])

In [5]:
symbols = list(currencies['symbol'][currencies['altcoin'] == False])

168

In [19]:
symbols

['AED',
 'AFN',
 'ALL',
 'AMD',
 'ANG',
 'AOA',
 'ARS',
 'AUD',
 'AWG',
 'AZN',
 'BAM',
 'BBD',
 'BDT',
 'BGN',
 'BHD',
 'BIF',
 'BMD',
 'BND',
 'BOB',
 'BRL',
 'BSD',
 'BTN',
 'BWP',
 'BYN',
 'BZD',
 'CAD',
 'CDF',
 'CHF',
 'CLF',
 'CLP',
 'CNH',
 'CNY',
 'COP',
 'CRC',
 'CUC',
 'CUP',
 'CVE',
 'CZK',
 'DJF',
 'DKK',
 'DOP',
 'DZD',
 'EGP',
 'ERN',
 'ETB',
 'EUR',
 'FJD',
 'FKP',
 'GBP',
 'GEL',
 'GGP',
 'GHS',
 'GIP',
 'GMD',
 'GNF',
 'GTQ',
 'GYD',
 'HKD',
 'HNL',
 'HRK',
 'HTG',
 'HUF',
 'IDR',
 'ILS',
 'IMP',
 'INR',
 'IQD',
 'IRR',
 'ISK',
 'JEP',
 'JMD',
 'JOD',
 'JPY',
 'KES',
 'KGS',
 'KHR',
 'KMF',
 'KPW',
 'KRW',
 'KWD',
 'KYD',
 'KZT',
 'LAK',
 'LBP',
 'LKR',
 'LRD',
 'LSL',
 'LYD',
 'MAD',
 'MDL',
 'MGA',
 'MKD',
 'MMK',
 'MNT',
 'MOP',
 'MRU',
 'MUR',
 'MVR',
 'MWK',
 'MXN',
 'MYR',
 'MZN',
 'NAD',
 'NGN',
 'NIO',
 'NOK',
 'NPR',
 'NZD',
 'OMR',
 'PAB',
 'PEN',
 'PGK',
 'PHP',
 'PKR',
 'PLN',
 'PYG',
 'QAR',
 'RON',
 'RSD',
 'RUB',
 'RWF',
 'SAR',
 'SBD',
 'SCR',
 'SDG',


In [34]:
currencies_not_included = ['USD'] #Which symbols to not run
symbols = list(currencies['symbol'][currencies['altcoin'] == False]) #Remove other cryptocurrencies
symbols = [e for e in symbols if e not in currencies_not_included] #Remove symbols not to run
start_tid = 56510558
end_tid = 56000000 #6000000 corresponds to Feb 2017



for currency in symbols:

    print(currency)
    start = timer()

    df = get_trades(start_tid, end_tid, currency)
    path = '../temporary/trades_' + str(currency) + '.csv'
    end = timer()
    time = end - start
    print(time)

    df.to_csv(path, index = False)

AED
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (0 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
2.707579899999473
AFN
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
4.398387300001559
ALL
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
6.884247099998902
AMD
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.

AttributeError: 'KeyError' object has no attribute 'response'

NameError: name 'gateway' is not defined

In [7]:
df

,date,tid,price,amount,Currency
0,1664330110,56510558,18029.58,0.03189204,USD
1,1664330110,56510556,21699.47,0.00553009,USD
2,1664330222,56510553,20045.22,0.00249436,USD
3,1664329958,56510549,18550.49,0.00307000,USD
4,1664330004,56510546,20549.77,0.00291974,USD
...,...,...,...,...,...
3427626,1487825175,5999834,1276.60,0.02350000,USD
3427627,1487825822,5999831,2478.31,0.32280000,USD
3427628,1487825126,5999830,1438.85,0.02780000,USD
3427629,1487825394,5999827,1490.51,0.03690000,USD


In [5]:
datetime.fromtimestamp(1664127368)

datetime.datetime(2022, 9, 25, 11, 36, 8)

In [25]:
#See most recent trades
t = requests.get('https://localbitcoins.com/bitcoincharts/AED/trades.json')
jprint(t.json())

[
    {
        "amount": "0.11759495",
        "date": 1664391054,
        "price": "71431.64",
        "tid": 56513570
    },
    {
        "amount": "0.03115198",
        "date": 1664385819,
        "price": "64201.38",
        "tid": 56513107
    },
    {
        "amount": "0.00878960",
        "date": 1664376465,
        "price": "81914.99",
        "tid": 56512310
    },
    {
        "amount": "0.00733217",
        "date": 1664367076,
        "price": "62600.84",
        "tid": 56511709
    },
    {
        "amount": "0.01429646",
        "date": 1664366941,
        "price": "61553.70",
        "tid": 56511687
    },
    {
        "amount": "0.00101703",
        "date": 1664328207,
        "price": "98325.52",
        "tid": 56510490
    },
    {
        "amount": "0.23377724",
        "date": 1664278646,
        "price": "72718.80",
        "tid": 56507029
    },
    {
        "amount": "0.00096337",
        "date": 1664262787,
        "price": "103802.28",
        "tid": 56506